#### Dense Passage Retrieval (DPR)

We saw how how to use the TFIDF representation for passages to perform retreival. One major problem with this kind of sparse vector representation is that if the query words don't exactly match any words from the relavant passages, then the retreival system will not be able to find those passages (because of zero cosine similarity between the query and passage vectors). To solve this problem, we can use some form of dense vector representation that can generalize better and can match a query to relevant documents even when there may not be exact matching words from the query in the passage. 

In the DPR model, we have `bi-encoders`, i.e. two separate BERT networks, a `query encoder` and a `pasage encoder`, which learn to map queries and passages respectively into a dense vector space in which the similarity between a query vector and it's corresponding relevant passage(s) is maximized. We use the output for the `[CLS]` token from each encoder as the dense vector representation. 

The bi-encoders are jointly trained using a supervised classification task where each input instance is a tuple $(q_i, p_i^{+}, p_{i,1}^{-}, ...,p_{i,n}^{-})$ where $q_i$ is a query, $p_i^{+}$ is a rlevant/positive passage and each of the $n$ $p_{i,j}^{-}$ are irrelevant/negative documents. Then we use the query encoder to compute the dense vector representation for the query $E_{Q}(q)$ and use the passage encoder for all the passages $E_P(p)$. Then we compute similarity scores between the query vector and each passage vector: $sim(q_i, p)$ for $p \in \{p_i^{+}, p_{i,1}^{-}, ...,p_{i,n}^{-}\}$. We can interpret these similarity scores as unnormalized logits for $(n+1)$ different class labels. With this interpretaion, we can define $sim(q_i, p_i^{+})$ as the logit for the "correct\ground truth class" and then simply use the `softmax cross-entropy/negative log-likelihood loss` function:

$L(q_i, p_i^{+}, p_{i,1}^{-}, ...,p_{i,n}^{-}) = -\log \frac{exp(sim(q_i, p_i^{+}))}{exp(sim(q_i, p_i^{+})) + \sum_{j=1}^n exp(sim(q_i, p_{i,j}^{-}))}$

Note that $[exp(sim(q_i, p_i^{+}), exp(sim(q_i, p_{i,1}^{-}),..., exp(sim(q_i, p_{i,n}^{-})]$ represents a probability distrbution and minimizing the loss function pushes $exp(sim(q_i, p_i^{+}))$ towards 1 and pushes the $exp(sim(q_i, p_{i,j}^{-}))$ towards zero, which allows us to achieve the dense vector space in which a query vector is maximally similar to the positive passage vector and dis-similar to the negative passages. We also use the simple `dot product` as our similarity metric.

For the SQuAD dataset, we already have given question, context passage pairs. Now we need to somehow choose negative passages for each pair. For training efficieny, we can use a simple trick. Given that we have a minibatch of $B$ such (question, context passage) pairs, then for each pair, we can simply just assign the passages from the other $B-1$ pairs as the negatives. Then we can compute the pair-wise dot product between every question-passgae pair with a single matrix multiplication. So given a matrix $Q$ of shape $(B,d)$ containing the batch of query vectors (where $d$ is the hidden dimensions of the encoded vectors) and a matrix $P$ of the same shape containing the batch of passages, we can compute the matrix $QP^T$ whose $(i,j)th$ entry given us the dot product between the ith question and the jth passage. So the $ith$ diagonal entry in this matrix is the dot product between $ith$ question and its corresponding positive passage and all other elements from that row are dot products with the negative passages. Then by taking the softmax of each row of this matrix, we can compute the total loss for the batch by just summing up the negative log of the terms along the diagonal. In addition to training efficiency, the other huge advantage of this technique is that the dataset will be shuffled before each epoch so that each question-positive passage pair will always get a different sample set of negative passages and therefore we effectively get a very large set of negatives per pair.

We wil use two DistilBERT models for our bi-encoders (two BERTs probably won't fit on my GPU and DistilBERT is half the size and performs almost as well as BERT anyway).


In [1]:
import torch
from transformers import DistilBertModel, DistilBertTokenizerFast
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import csv
import random
from tqdm import tqdm
import psutil
import json
import wandb
import os

wandb.login()
print(torch.cuda.is_available())

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanzids. Use `wandb login --relogin` to force relogin


True


#### Let's define our Bi-encoder model first.

In [2]:
class BERTBiEncoder(torch.nn.Module):
    def __init__(self, dropout_rate=0.1):
        super().__init__()
        # load pretrained BERT model
        self.query_encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.passage_encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = torch.nn.Dropout(dropout_rate)

        for param in self.query_encoder.parameters():
            param.requires_grad = True
        for param in self.passage_encoder.parameters():
            param.requires_grad = True

    def forward(self, query_idx, query_attn_mask, passage_idx, passage_attn_mask):
        # compute BERT encodings, extract the `[CLS]` encoding (first element of the sequence), apply dropout        
        passage_output = self.passage_encoder(passage_idx, attention_mask=passage_attn_mask)
        passage_enc = self.dropout(passage_output.last_hidden_state[:,0]) # shape: (batch_size, hidden_size)
        query_output = self.query_encoder(query_idx, attention_mask=query_attn_mask)
        query_enc = self.dropout(query_output.last_hidden_state[:, 0]) # shape: (batch_size, hidden_size)
        # compute similarity score matrix
        scores = torch.mm(query_enc, passage_enc.transpose(0, 1)) # shape: (batch_size, batch_size)
        # compute cross-entropy loss
        loss = F.cross_entropy(scores, torch.arange(len(scores)).to(scores.device))
        return scores, loss
    
    @ torch.no_grad()
    def encode_queries(self, query_idx, query_attn_mask):
        self.eval()
        query_output = self.query_encoder(query_idx, attention_mask=query_attn_mask)
        query_enc = self.dropout(query_output.last_hidden_state[:, 0])
        return query_enc

    @ torch.no_grad()
    def encode_passages(self, passage_idx, passage_attn_mask):
        self.eval()
        passage_output = self.passage_encoder(passage_idx, attention_mask=passage_attn_mask)
        passage_enc = self.dropout(passage_output.last_hidden_state[:,0]) # shape: (batch_size, hidden_size)
        return passage_enc

# training loop
def train(model, optimizer, train_dataloader, val_dataloader, scheduler=None, device="cpu", num_epochs=10, accumulation_steps=1, val_every=100, save_every=None, log_metrics=None):
    avg_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    model.train()
    # reset gradients
    optimizer.zero_grad()
    for epoch in range(num_epochs):
        num_correct = 0
        num_total = 0
        pbar = tqdm(train_dataloader, desc="Epochs")
        for i, batch in enumerate(pbar):
            query_idx, query_attn_mask, passage_idx, passage_attn_mask = batch
            # move batch to device
            query_idx, query_attn_mask, passage_idx, passage_attn_mask = query_idx.to(device), query_attn_mask.to(device), passage_idx.to(device), passage_attn_mask.to(device)
            # forward pass
            scores, loss = model(query_idx, query_attn_mask, passage_idx, passage_attn_mask )
            # backward pass
            loss.backward()
            # apply gradient step 
            if (i+1) % accumulation_steps == 0:
                # optimizer step
                optimizer.step()
                # reset gradients
                optimizer.zero_grad()
   
            avg_loss = 0.9* avg_loss + 0.1*loss.item()
            B, _ = query_idx.shape
            y_pred = scores.argmax(dim=-1).view(-1) # shape (B,)
            targets = torch.arange(B).to(device) # shape (B,)
            num_correct += (y_pred.eq(targets.view(-1))).sum().item()      
            num_total += B
            train_acc = num_correct / num_total        

            if val_every is not None:
                if i%val_every == 0:
                    # compute validation loss
                    val_loss, val_acc = validation(model, val_dataloader, device=device)
                    pbar.set_description(f"Epoch {epoch + 1}, EMA Train Loss: {avg_loss:.3f}, Train Accuracy: {train_acc: .3f}, Val Loss: {val_loss: .3f}, Val Accuracy: {val_acc: .3f}")  

            pbar.set_description(f"Epoch {epoch + 1}, EMA Train Loss: {avg_loss:.3f}, Train Accuracy: {train_acc: .3f}, Val Loss: {val_loss: .3f}, Val Accuracy: {val_acc: .3f}")  

            if log_metrics:
                metrics = {"Batch loss" : loss.item(), "Moving Avg Loss" : avg_loss, "Val Loss": val_loss}
                log_metrics(metrics)

        # run optimizer step for remainder batches
        if len(train_dataloader) % accumulation_steps != 0:
            optimizer.step()
            optimizer.zero_grad()

        if scheduler is not None:
            scheduler.step()

        if save_every is not None:
            if (epoch+1) % save_every == 0:
                save_model_checkpoint(model, optimizer, epoch, avg_loss)


def validation(model, val_dataloader, device="cpu"):
    model.eval()
    val_losses = torch.zeros(len(val_dataloader))
    with torch.no_grad():
        num_correct = 0
        num_total = 0
        for i,batch in enumerate(val_dataloader):
            query_idx, query_attn_mask, passage_idx, passage_attn_mask = batch
            query_idx, query_attn_mask, passage_idx, passage_attn_mask = query_idx.to(device), query_attn_mask.to(device), passage_idx.to(device), passage_attn_mask.to(device)
            scores, loss = model(query_idx, query_attn_mask, passage_idx, passage_attn_mask)
            B, _ = query_idx.shape
            y_pred = scores.argmax(dim=-1).view(-1) # shape (B,)
            targets = torch.arange(B).to(device) # shape (B,)
            num_correct += (y_pred.eq(targets.view(-1))).sum().item()      
            num_total += B
            val_losses[i] = loss.item()
    model.train()
    val_loss = val_losses.mean().item()
    val_accuracy = num_correct / num_total
    return val_loss, val_accuracy

def save_model_checkpoint(model, optimizer, epoch=None, loss=None, filename=None):
    # Save the model and optimizer state_dict
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    # Save the checkpoint to a file
    if filename:
        torch.save(checkpoint, filename)
    else:
        torch.save(checkpoint, 'dpr_checkpoint.pth')
    print(f"Saved model checkpoint!")


def load_model_checkpoint(model, optimizer, filename=None):
    if filename:
        checkpoint = torch.load(filename)
    else:
        checkpoint = torch.load('dpr_checkpoint.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.train()
    print("Loaded model from checkpoint!")
    return model, optimizer          

#### Let's load up the SQuAD v1 data

In [3]:
# load the train and dev JSON documents
with open("train.json", "r") as train_file:
    squad_train = json.load(train_file)         
with open("dev.json", "r") as dev_file:
    squad_dev = json.load(dev_file) 

def get_passages(squad, num_titles=None):
    if num_titles is None:
        num_titles = len(squad['data'])
    # for each title, get passages and all corresponding questions from SQuAD train set
    passages = []
    questions = []
    num_questions = 0
    j = 0
    for i in range(num_titles):
        #print(f"Title# {i}: {squad['data'][i]['title']}, Number of passages: {len(squad['data'][i]['paragraphs'])}")
        for p in squad['data'][i]['paragraphs']:
            passages.append(p['context'])
            for q in p['qas']:
                if not q['is_impossible']:
                    questions.append((q,j))    
                    num_questions += 1
            j += 1
    print(f"Number of passages: {len(passages)}")
    print(f"Number of questions: {num_questions}")
    return passages, questions

passages_train, questions_train = get_passages(squad_train)
passages_val, questions_val = get_passages(squad_dev)

Number of passages: 19035
Number of questions: 86821
Number of passages: 1204
Number of questions: 5928


In [4]:
# need this because huggingface tokenizer is not thread safe when using it inside __getitem__ instead of dataloader collatefunction
os.environ["TOKENIZERS_PARALLELISM"] = "false"  #

class SquadDataset(Dataset):
    def __init__(self, passages, questions, block_size=128):
        self.passages = passages
        self.questions = questions
        self.tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
        self.block_size = block_size
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        # get the question and context passage
        q = self.questions[idx][0]
        passage_idx = self.questions[idx][1]
        question = q['question']
        passage = self.passages[passage_idx]
        # tokenize the context passage
        passage_encoding = self.tokenizer.encode_plus(passage, add_special_tokens=False, return_offsets_mapping=False, return_attention_mask=False, return_token_type_ids=False)
        passage_idx = passage_encoding['input_ids']
        # tokenize the question
        question_encoding = self.tokenizer.encode_plus(question, add_special_tokens=False, return_offsets_mapping=False, return_attention_mask=False, return_token_type_ids=False)
        question_idx = question_encoding['input_ids']

        # get answer span start and end character positions, for multiple answers, we will only use the first answer
        first_answer_idx = 0
        answer_start_char = q['answers'][first_answer_idx]['answer_start']
        answer_end_char = answer_start_char + len(q['answers'][first_answer_idx]['text'])
        # convert char positions to token positions
        answer_start_token = passage_encoding.char_to_token(answer_start_char)
        answer_end_token = passage_encoding.char_to_token(answer_end_char-1)

        # select a window size so that the passage sequence will be no longer than the block size
        window_size_tokens = self.block_size - 2 # 2 special tokens ([CLS], [SEP])
        # now create a window around the answer span, pick the window start position randomly
        window_start_min = max(0, answer_end_token - window_size_tokens + 1)
        window_start_max = min(answer_start_token, max(0,len(passage_idx) - window_size_tokens)) # we want to make the window as large as possible, but not go over the end of the context
        window_start = random.randint(window_start_min, window_start_max)
        window_end = window_start + window_size_tokens        
        # select window of passage tokens
        passage_window_tokens = passage_idx[window_start:window_end]

        # create padded query and passage sequences
        question_idx = [self.tokenizer.cls_token_id] + question_idx + [self.tokenizer.sep_token_id]
        question_idx = question_idx + [self.tokenizer.pad_token_id]*(self.block_size-len(question_idx))
        passage_idx = [self.tokenizer.cls_token_id] + passage_window_tokens + [self.tokenizer.sep_token_id]
        passage_idx = passage_idx + [self.tokenizer.pad_token_id]*(self.block_size-len(passage_idx))

        # make sure the passage sequence and query sequences are not longer than max_length
        if len(question_idx) > self.block_size or len(passage_idx) > self.block_size:
            raise Exception(f"Passage sequence length {len(passage_idx)} or question sequence length {len(question_idx)} is longer than max_length {self.block_size}!")
        
        # create attention masks
        question_attn_mask = [1 if idx != self.tokenizer.pad_token_id else 0 for idx in question_idx]
        passage_attn_mask  = [1 if idx != self.tokenizer.pad_token_id else 0 for idx in passage_idx]

        # convert to tensors
        question_idx = torch.tensor(question_idx)
        question_attn_mask = torch.tensor(question_attn_mask)
        passage_idx = torch.tensor(passage_idx)
        passage_attn_mask = torch.tensor(passage_attn_mask)

        return question_idx, question_attn_mask, passage_idx, passage_attn_mask

In [5]:
block_size = 192
train_dataset = SquadDataset(passages_train, questions_train, block_size=block_size)
val_dataset = SquadDataset(passages_val, questions_val, block_size=block_size)

#### Now train the model

In [6]:
B = 16
DEVICE = "cuda"
learning_rate = 5e-6

train_dataloader = DataLoader(train_dataset, batch_size=B, shuffle=True, pin_memory=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=B, shuffle=True, pin_memory=True, num_workers=2)

# model with finetuning disabled
model = BERTBiEncoder().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.95)
model, optimizer = load_model_checkpoint(model, optimizer)

num_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters in transformer network: {num_params/1e6} M")
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")


Loaded model from checkpoint!
Total number of parameters in transformer network: 132.72576 M
RAM used: 3363.17 MB


In [8]:
# create a W&B run
run = wandb.init(
    project="DPR", 
    config={
        "bi-encoder model": "MobileBERT",
        "learning_rate": learning_rate, 
        "epochs": 5,
        "batch_size": B, 
        "corpus": "SQuAD v1"},)   

def log_metrics(metrics):
    wandb.log(metrics)

In [15]:
train(model, optimizer, train_dataloader, val_dataloader, device=DEVICE, num_epochs=2, save_every=2, val_every=1000, log_metrics=log_metrics) 

Epochs:   0%|          | 0/5427 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors
Epoch 1, EMA Train Loss: 0.019, Train Accuracy:  0.991, Val Loss:  0.105, Val Accuracy:  0.966:   1%|          | 56/5427 [00:50<26:20,  3.40it/s]  Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors
Epoch 1, EMA Train Loss: 0.026, Train Accuracy:  0.991, Val Loss:  0.105, Val Accuracy:  0.966:   1%|▏         | 69/5427 [00:54<25:57,  3.44it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Run

Saved model checkpoint!


In [12]:
#save_model_checkpoint(model, optimizer)

Saved model checkpoint!


In [16]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: ERROR Control-C detected -- Run data was not synced


### Now that the model has been trained, let's precompute the encodings for all passages across the training and validation set, then evaluate the top-k retreival accuracy.

In [7]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [8]:
passages = passages_train + passages_val
questions = questions_train + questions_val

b_size = 196-2

# first we create fixed sized blocks from the passages, we will only use the validation set for this
passage_blocks = []
passage_blocks_enc = []
passage_blocks_index = {}
j = 0
for k, p in enumerate(passages):
    #print(p)
    p_enc = tokenizer.encode_plus(p, add_special_tokens=False, return_offsets_mapping=False, return_attention_mask=False, return_token_type_ids=False)['input_ids']
    # split p_enc into blocks of size b_size
    for i in range(0, len(p_enc), b_size):
        p_enc_block = [tokenizer.cls_token_id] + p_enc[i:i+b_size] + [tokenizer.sep_token_id]  
        # apply padding to the end of the block
        p_enc_block = p_enc_block + [tokenizer.pad_token_id]*(b_size-len(p_enc_block))  
        passage_blocks_enc.append(p_enc_block)
        passage_blocks.append(tokenizer.decode(p_enc[i:i+b_size]))
        passage_blocks_index[j] = k
        j += 1    

Token indices sequence length is longer than the specified maximum sequence length for this model (718 > 512). Running this sequence through the model will result in indexing errors


In [9]:
def create_attention_mask(p_enc_block):
    # create attention masks
    attn_mask  = [1 if idx != tokenizer.pad_token_id else 0 for idx in p_enc_block]
    return attn_mask

def encode_query(query):
    query_enc = tokenizer.encode_plus(query, add_special_tokens=False, return_offsets_mapping=False, return_attention_mask=False, return_token_type_ids=False)['input_ids']
    # apply padding to the end of the block
    query_enc = [tokenizer.cls_token_id] + query_enc + [tokenizer.sep_token_id] + [tokenizer.pad_token_id]*(b_size-len(query_enc)-2)  
    return query_enc    

In [11]:
# pre-compute the dense encodings for all passages
passage_blocks_vectors = []
pbar = tqdm(passage_blocks_enc, desc="Question#")
for p in pbar:
    passage_blocks_enc_tensor = torch.tensor(p).unsqueeze(0).to(DEVICE)
    attn_mask = torch.tensor(create_attention_mask(p)).unsqueeze(0).to(DEVICE)
    passage_vector = model.encode_passages(passage_blocks_enc_tensor, attn_mask)
    passage_blocks_vectors.append(passage_vector.squeeze(0))

# now stack up the passage vectors into a matrix
passage_blocks_vectors = torch.stack(passage_blocks_vectors)    

Question#: 100%|██████████| 24502/24502 [09:30<00:00, 42.92it/s]


In [14]:
def retreive_passages(query, k=5):
    # encode the query
    q_enc = encode_query(query)
    # compute the query vector
    q_enc_tensor = torch.tensor(q_enc).unsqueeze(0).to(DEVICE)
    attn_mask = torch.tensor(create_attention_mask(q_enc)).unsqueeze(0).to(DEVICE)
    q_vector = model.encode_queries(q_enc_tensor, attn_mask)
    # compute the similarity scores
    scores = torch.mm(passage_blocks_vectors, q_vector.view(-1,1))
    # get the top k scores
    topk_scores, topk_indices = torch.topk(scores.view(-1), k=k)
    # get topk passages
    topk_passages = [passage_blocks[i] for i in topk_indices]
    main_passages_idx = [passage_blocks_index[i.item()] for i in topk_indices]
    return topk_scores, topk_passages, main_passages_idx    
        

In [32]:
# test on some randomly selected questions
question_idx = random.randint(0, len(questions)-1)
query = questions[question_idx][0]['question']
gold_passage_idx = questions[question_idx][1]

passage = passages[gold_passage_idx]
print(f"Question: {query}")
print(f"Gold Passage Idx# {gold_passage_idx}: {passage}")
print(f"Answer: {questions[question_idx][0]['answers'][0]['text']}")

# get topk passages
topk_scores, topk_passages, main_passages_idx = retreive_passages(query, k=5)
if(gold_passage_idx in main_passages_idx):
    print("Gold passage found in topk passages!")

print(f"\nTopk passages:")
for i in range(len(topk_passages)):
    print(f"\nPassage#{i}: {topk_passages[i]}")
    print(f"Main Passage Index: {main_passages_idx[i]}")
    print(f"Score: {topk_scores[i]}")


Question: Who is Beyoncé's biggest musical influence?
Gold Passage Idx# 38: Beyoncé names Michael Jackson as her major musical influence. Aged five, Beyoncé attended her first ever concert where Jackson performed and she claims to have realised her purpose. When she presented him with a tribute award at the World Music Awards in 2006, Beyoncé said, "if it wasn't for Michael Jackson, I would never ever have performed." She admires Diana Ross as an "all-around entertainer" and Whitney Houston, who she said "inspired me to get up there and do what she did." She credits Mariah Carey's singing and her song "Vision of Love" as influencing her to begin practicing vocal runs as a child. Her other musical influences include Aaliyah, Prince, Lauryn Hill, Sade Adu, Donna Summer, Mary J. Blige, Janet Jackson, Anita Baker and Rachelle Ferrell.
Answer: Michael Jackson
Gold passage found in topk passages!

Topk passages:

Passage#0: beyonce has stated that she is personally inspired by us first lady 

Retrieve passages for every question from validation set and compute topk accuracy

In [22]:
def compute_topk_accuracy(k=5, num_questions=1000): 
    num_correct = 0
    num_total = 0
    idxs = random.sample(range(0, len(questions)), num_questions)
    pbar = tqdm(idxs, desc="Question#")
    for question_idx in pbar:
        query = questions[question_idx][0]['question']
        gold_passage_idx = questions[question_idx][1]
        # get topk passages
        topk_scores, topk_passages, main_passages_idx = retreive_passages(query, k)
        if(gold_passage_idx in main_passages_idx):
            num_correct += 1
        num_total += 1    
        pbar.set_description(f"Top-{k} Accuracy: {num_correct/num_total:.3f}")  

    print(f"Top-{k} accuracy = {num_correct/num_total:.3f}")

In [35]:
compute_topk_accuracy(k=10, num_questions=10000)

Top-10 Accuracy: 0.800: 100%|██████████| 10000/10000 [06:19<00:00, 26.36it/s]

Top-10 accuracy = 0.800


Note that the top-10 retreival accuracy of this DPR model is comparable to BM25. Also note that since we've broken the passges up into smaller fixed size blocks, this might give DPR a slight advantage. Also since we're working with a small number of passages, DPR retreival is faster than BM25, but this isn't typically true at larger scales.

Now, let's try passage retrieval with our own made up queries.

In [58]:
query = "Which decade did the heavy metal genre emerge?"
topk_scores, topk_passages, main_passages_idx = retreive_passages(query, k=20)

print(f"\nTopk passages:")
for i in range(len(topk_passages)):
    print(f"\nPassage#{i}: {topk_passages[i]}")
    print(f"Main Passage {main_passages_idx[i]}: {passages[main_passages_idx[i]]}")
    print(f"Score: {topk_scores[i]}")



Topk passages:

Passage#0: in the late 1960s, the term heavy metal was used interchangeably with hard rock, but gradually began to be used to describe music played with even more volume and intensity. while hard rock maintained a bluesy rock and roll identity, including some swing in the back beat and riffs that tended to outline chord progressions in their hooks, heavy metal's riffs often functioned as stand - alone melodies and had no swing in them. heavy metal took on " darker " characteristics after black sabbath's breakthrough at the beginning of the 1970s. in the 1980s it developed a number of subgenres, often termed extreme metal, some of which were influenced by hardcore punk, and which further differentiated the two styles. despite this differentiation, hard rock and heavy metal have existed side by side, with bands frequently standing on the boundary of, or crossing between, the genres.
Main Passage 6115: In the late 1960s, the term heavy metal was used interchangeably with 